In [ ]:
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
   
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

class ObjectDetector(object):
   
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
       
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
   
    def __call__(self, *inputs):
        return self.execute(*inputs)

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

In [ ]:
#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
   
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
   
    def __init__(self):
        super(Camera, self).__init__()
       
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
       
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
       
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()  
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()          
        self.depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap  

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
           
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable

            depth_frame = frames.get_depth_frame() #get the depth image          
            self.depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
   
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread      

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

In [ ]:
### PROCESS

# if first frame
#   if there are detections
#       find nearest person
#       find coordinates of the center of their bounding box
#       move accordingly towards the boudning box
#
#   else: (no detections)
#       sleep for a second
#
# else: (not first frame)
#   if there are detections
#       if any detection is at least 0.65 (correlation) similar to the person being followed in previous frame
#           choose the first detection that satisfies this
#           find the center of their bounding box
#           move accordingly towards the boudning box
#
#       else: (no detections are at least 0.65 similar to the person from previous frame)
#           choose the nearest detection [0]
#           find the center of their bounding box
#           move accordingly towards the bounding box
#
#   else: (no detections)
#       sleep for a second

import ipywidgets.widgets as widgets
from IPython.display import display, HTML

# cent = (width, height)
width = 640
height = 480

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
display(widgets.VBox([
    widgets.HBox([image_widget,]),
    label_widget
]))

import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()
runs = 0 # keeps track of frame

def crop_bb(image, bbox):
    full_coords = [width*bbox[0], height*bbox[1], width*bbox[2], height*bbox[3]]
    full_coords = [round(_) for _ in full_coords]

    cropped_person = image[full_coords[1]:full_coords[3], full_coords[0]:full_coords[2]]

    return cropped_person

def calc_hist(image):
    hist = cv2.calcHist([image], [0], None, [256], [0,255])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def processing(change):
    global runs
    image = change['new']
     
    imgsized= cv2.resize(image,(300,300))
    # compute all detected objects
    detections = model(imgsized)
   
    print(detections[0])
    if runs == 0:
        # checks to see if this is the first frame
        matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
        distance_human = 0
        # finds all the detections in the frame
        if matching_detections:
            # if any person has been detected, ...
            runs += 1
            first_person = matching_detections[0]
            # gets the closest detected instance to the camera
            bbox = first_person['bbox']
            cent =[(int(width * bbox[0])+int(width * bbox[2]))//2,(int(height * bbox[1])+int(height * bbox[3]))//2]
            # finds the center of the bounding box of an instance
            distance_human = camera.depth_image[cent[0],cent[1]]
            # calculates instance distance from robot
            cv2.putText(image, str(distance_human), (cent[0], cent[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            movement(cent, distance_human)

            cropped_boundingbox = crop_bb(image, bbox)
            old_histogram = calc_hist(cropped_boundingbox)
        else:
            # if no detections are found, the robot waits
            time.sleep(1.0)
    else:
        # if it is not on the first frame
        matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
        # finds all detected isntances
        if matching_detections:
            # if at least one person has been detected
            person_index = same_person(matching_detections, old_histogram)
            # finds the index (in matching_detections) of the first detection that is considered similar to the person being followed
            person_bb = matching_detections[person_index]
            cent = [(int(width * person_bb[0])+int(width * person_bb[2]))//2,(int(height * person_bb[1])+int(height * person_bb[3]))//2]
            # finds center of bounding box
            distance_human = camera.depth_image[cent[0],cent[1]]
            # finds distance of instance from robot
            cv2.putText(image, str(distance_human), (cent[0], cent[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            movement(cent, distance_human)
            crop_person = crop_bb(image, person_bb)
            old_histogram = calc_hist(crop_person)
        else:
            time.sleep(1.0)
    image_widget.value = bgr8_to_jpeg(image)      



def same_person(matching_detections, old_histogram, image):
    """ 
        Iterates through all detections to see if any of the detections are similar enough to the perosn being followed from the last frame to be considered the same

        matching_detections - a dictionary containing information on every detected instance
        old_histogram - the normalised image histogram of the person in the previous frame
        image - the frame to crop bounding boxes from
    """
    
    for idx, i in enumerate(matching_detections):
        bbox = i['bbox']
        cropped_person = crop_bb(image, bbox)
        # crops the instances bounding box from the frame
    
        new_histogram = calc_hist(cropped_person)
        # calculates the image histogram of the cropped boounding box
        
        if cv2.compareHist(new_histogram, old_histogram, cv2.HISTCMP_CORREL) > 0.65:
            # checks to see if each bounding box's histogram is similar enough to the person instance to be considered the same instance
            return i
    return 0
        


    
def movement(cent, distance_human):
    """
        Controls how the robot will move, taking into consideration how off centre the target instance is and how far away it is
        
        cent - center of the instance (to be followed) bounding box
        distance_human - distance of instance from robot
    """
    threshold = 20
    distance = width // 2 - cent[0] # distance from center line to center of bbox
    prev_dir = 'n/a'
    base_speed = 0.3 # Default speed
    if distance_human >1000:
        prev_dir = 'forward'
        robot.forward(0.3)
    elif distance > threshold:
        # turn left
        speed = base_speed * distance # calculates movements speed dependent on position of target 
        robot.left(speed)
        prev_dir = 'left'
    elif distance < -threshold:
        speed = base_speed * -distance # calculates movements speed dependent on position of target 
        #  turn right
        robot.right(speed)
        prev_dir = 'right'
    else:
        if prev_dir == 'forward':
            robot.forward(0.3)
        elif prev_dir == 'left':
            robot.left(0.2)
        elif prev_dir == 'right':
            robot.right(0.2)
        else:
            robot.stop()
        

   
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
camera.observe(processing, names='color_value')
   

In [4]:
#  for det in matching_detections:
# #         bbox = det['bbox']
# #         cent =[(int(width * bbox[0])+int(width * bbox[2]))//2,(int(height * bbox[1])+int(height * bbox[3]))//2]
# #         cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)

# #         if cent[0] < height and cent[1] < width:
       
# #             distance_human = camera.depth_image[cent[0],cent[1]]
# #             cv2.putText(image, str(distance_human), (cent[0], cent[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

# #             width_bbox0 = int(width * bbox[0])
# #             width_bbox2 = int(width * bbox[2])
# #             height_bbox1 = int(height * bbox[1])
# #             height_bbox2 = int(height * bbox[3])

# #             print('widthb0 =', width_bbox0)
# #             print('wb2 =', width_bbox2)
# #             print('hb1=', height_bbox1)
# #             print('hb2=', height_bbox2)
       
       
#     if len(matching_detections)==0:
#         robot.stop()
#     else:
#         if distance_human >1000:
#             robot.forward(0.5)
#         elif distance_human >400:
#             robot.forward(0.3)
#         else:
#             robot.stop()
       
#     image_widget.value = bgr8_to_jpeg(image)
   
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
# camera.observe(processing, names='color_value')

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [5]:
camera.unobserve_all()
camera.stop()
time.sleep(1.0)
robot.stop()

[]
[]
